In [1]:
import bayes3d as b
import os
import glob
import nvsmi
from tqdm import tqdm
import numpy as np
import jax
import functools
import jax.numpy as jnp
from jax.debug import print as jprint
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL

In [2]:
def display_mcs_video(images, framerate=30):
    frames = [b.get_rgb_image(images[i].rgb) for i in range(len(images))]
    if type(frames[0]) == PIL.Image.Image:
      frames = [np.array(frames[i]) for i in range(len(frames))]
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
        im.set_data(frame)
        return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                   interval=interval, blit=True, repeat=True)
    return HTML(anim.to_html5_video())

def get_gpu_mem():
    if len(nvsmi.get_gpu_processes()) > 0:
        return nvsmi.get_gpu_processes()[0].used_memory
    else:
        return 0.0

class RGBD(object):
    def __init__(self, rgb, depth, camera_pose, intrinsics, segmentation=None):
        """RGBD Image
        
        Args:
            rgb (np.array): RGB image
            depth (np.array): Depth image
            camera_pose (np.array): Camera pose. 4x4 matrix
            intrinsics (b.camera.Intrinsics): Camera intrinsics
            segmentation (np.array): Segmentation image
        """
        self.rgb = rgb
        self.depth = depth
        self.camera_pose = camera_pose
        self.intrinsics = intrinsics
        self.segmentation  = segmentation

def splice_image_parallel(rendered_object_image, obs_image_complement):
    keep_masks = jnp.logical_or(
        (rendered_object_image[:,:,:,2] <= obs_image_complement[None, :,:, 2]) * 
        rendered_object_image[:,:,:,2] > 0.0
        ,
        (obs_image_complement[:,:,2] == 0)[None, ...]
    )[...,None]
    rendered_images = keep_masks * rendered_object_image + (1.0 - keep_masks) * obs_image_complement
    return rendered_images

In [3]:
@functools.partial(
    jnp.vectorize,
    signature='(m)->()',
    excluded=(1,2,3,4,),
)
def gausssian_mixture_per_pixel(
    ij,
    data_xyz: jnp.ndarray,
    model_xyz: jnp.ndarray,
    filter_size: int,
    r
):
    dists = data_xyz[ij[0], ij[1], :3] - jax.lax.dynamic_slice(model_xyz, (ij[0], ij[1], 0), (2*filter_size + 1, 2*filter_size + 1, 3))
    probs = (jax.scipy.stats.norm.pdf(dists, loc=0, scale=r)).prod(-1).sum()
    return probs
    
def gaussian_mixture_image(
    obs_xyz: jnp.ndarray,
    rendered_xyz: jnp.ndarray,
    r
):
    filter_size = 3
    num_latent_points = obs_xyz.shape[1] * obs_xyz.shape[0]
    rendered_xyz_padded = jax.lax.pad(rendered_xyz,  -100.0, ((filter_size,filter_size,0,),(filter_size,filter_size,0,),(0,0,0,)))
    jj, ii = jnp.meshgrid(jnp.arange(obs_xyz.shape[1]), jnp.arange(obs_xyz.shape[0]))
    indices = jnp.stack([ii,jj],axis=-1)
    probs = gausssian_mixture_per_pixel(indices, obs_xyz, rendered_xyz_padded, filter_size, r)
    return probs

gaussian_mixture_image_jit = jax.jit(gaussian_mixture_image)

def threedp3_likelihood(
    obs_xyz: jnp.ndarray,
    rendered_xyz: jnp.ndarray,
    r,
    outlier_prob,
    outlier_volume,
):
    num_latent_points = obs_xyz.shape[1] * obs_xyz.shape[0]
    probs = gaussian_mixture_image(obs_xyz, rendered_xyz, r)
    probs_with_outlier_model = probs * (1.0 - outlier_prob) / num_latent_points   + outlier_prob / outlier_volume
    return jnp.log(probs_with_outlier_model).sum()

threedp3_likelihood_parallel = jax.vmap(threedp3_likelihood, in_axes=(None, 0, None, None, None))
threedp3_likelihood_parallel_jit = jax.jit(threedp3_likelihood_parallel)
threedp3_likelihood_jit = jax.jit(threedp3_likelihood)
threedp3_likelihood_with_r_parallel_jit = jax.jit(
    jax.vmap(threedp3_likelihood_parallel, in_axes=(None, None, 0, None, None)),
)

In [4]:
def load_mcs_scene_data(scene_name = "passive_physics_shape_constancy_0001_01", scene_dir = "../../../../../physics_npzs"):
    images = np.load(scene_dir + "/{}.npz".format(scene_name),allow_pickle=True)["arr_0"]
    images = images[85:] # change this
    return images

In [5]:
def get_object_mask(point_cloud_image, segmentation, segmentation_ids):
    object_mask = jnp.zeros(point_cloud_image.shape[:2])
    object_ids = []
    for id in segmentation_ids:
        point_cloud_segment = point_cloud_image[segmentation == id]
        bbox_dims, pose = b.utils.aabb(point_cloud_segment)
        is_occluder = jnp.logical_or(jnp.logical_or(jnp.logical_or(jnp.logical_or(
            (bbox_dims[0] < 0.1),
            (bbox_dims[1] < 0.1)),
            (bbox_dims[1] > 1.1)),
            (bbox_dims[0] > 1.1)),
            (bbox_dims[2] > 2.1)
        )
        if not is_occluder:
            object_mask += (segmentation == id)
            object_ids.append(id)

    object_mask = jnp.array(object_mask) > 0
    return object_ids, object_mask

def prior(new_state, prev_poses, prev_prev_poses, bbox_dims, known_id):    
    score = 0.0
    new_position = new_state[:3,3]
    bottom_of_object_y = new_position[1] + bbox_dims[known_id][1]/2.0

    prev_position = prev_poses[known_id][:3,3]
    prev_prev_position = prev_prev_poses[known_id][:3,3]

    velocity_prev = (prev_position - prev_prev_position) * jnp.array([1.0, 1.0, 0.25])
    velocity_with_gravity = velocity_prev + jnp.array([-jnp.sign(velocity_prev[0])*0.01, 0.1, 0.0])

    velocity_with_gravity2 = velocity_with_gravity * jnp.array([1.0 * (jnp.abs(velocity_with_gravity[0]) > 0.1), 1.0, 1.0 ])
    velocity = velocity_with_gravity2

    pred_new_position = prev_position + velocity

    score = score + jax.scipy.stats.multivariate_normal.logpdf(
        new_position, pred_new_position, jnp.diag(jnp.array([0.02, 0.02, 0.02]))
    )
    score += -100.0 * (bottom_of_object_y > 1.5)
    return score

prior_jit = jax.jit(prior)
prior_parallel_jit = jax.jit(jax.vmap(prior, in_axes=(0, None,  None, None, None)))

WALL_Z = 14.5
FLOOR_Y = 1.45

In [8]:
class PhysicsServer():
    def __init__(self):
        pass

    def reset(self, intrinsics):
        self.original_intrinsics = intrinsics
        intrinsics = b.camera.scale_camera_parameters(self.original_intrinsics, 0.25)
        intrinsics = b.Intrinsics(
            intrinsics.height, intrinsics.width,
            intrinsics.fx,
            intrinsics.fy,
            intrinsics.cx,
            intrinsics.cy,
            intrinsics.near,
            WALL_Z
        )
        self.intrinsics = intrinsics
        
        self.ALL_OBJECT_POSES = [jnp.zeros((0, 4, 4))]
        self.renderer = b.Renderer(self.intrinsics)
        self.t = 0
        self.first_appearance = []

        dx  = 0.7
        dy = 0.7
        dz = 0.7
        gridding1 = b.utils.make_translation_grid_enumeration(
            -dx, -dy, -dz, dx, dy, dz, 21,15,15
        )
        self.gridding = [gridding1]
        self.plausibility = [0.0]
        self.violation_locations = [[]]
        self.images = []

    def update(self, image):
        
        self.t += 1
        self.images.append(image)

        t = self.t
        images = self.images
        intrinsics = self.intrinsics
        gridding = self.gridding
        renderer = self.renderer
        plausibility = self.plausibility[-1]

        ALL_OBJECT_POSES = self.ALL_OBJECT_POSES


        R_SWEEP = jnp.array([0.03])
        OUTLIER_PROB=0.05
        OUTLIER_VOLUME=1.0


        depth = b.utils.resize(image.depth, intrinsics.height, intrinsics.width)
        point_cloud_image = b.t3d.unproject_depth(depth, intrinsics)

        segmentation = b.utils.resize(image.segmentation, intrinsics.height, intrinsics.width)
        segmentation_ids = jnp.unique(segmentation)

        object_ids, object_mask = get_object_mask(point_cloud_image, segmentation, segmentation_ids)
        depth_complement = depth * (1.0 - object_mask) + intrinsics.far * (object_mask)
        point_cloud_image_complement = b.t3d.unproject_depth(depth_complement, intrinsics)

        OBJECT_POSES = jnp.array(ALL_OBJECT_POSES[t-1])
        for known_id in range(OBJECT_POSES.shape[0]):

            current_pose_estimate = OBJECT_POSES[known_id, :, :]

            for gridding_iter in range(len(gridding)):
                all_pose_proposals = [
                    jnp.einsum("aij,jk->aik", 
                        gridding[gridding_iter],
                        current_pose_estimate,
                    )
                ]
                if gridding_iter == 0:
                    for seg_id in object_ids:
                        _, center_pose = b.utils.aabb(point_cloud_image[segmentation==seg_id])
                        all_pose_proposals.append(
                            jnp.einsum("aij,jk->aik", 
                                gridding[gridding_iter],
                                center_pose,
                            )
                        )
                all_pose_proposals = jnp.vstack(all_pose_proposals)

                all_weights = []
                for batch in jnp.array_split(all_pose_proposals,4):
                    #########################################################################################
                    # jprint("Before render 1: {}", get_gpu_mem())
                    rendered_images = renderer.render_many(batch[:,None,...], jnp.array([known_id]))[...,:3]
                    # jprint("After render 1: {}", get_gpu_mem())
                    rendered_images_spliced = splice_image_parallel(rendered_images, point_cloud_image_complement)
                    weights = threedp3_likelihood_with_r_parallel_jit(
                        point_cloud_image, rendered_images_spliced, R_SWEEP, OUTLIER_PROB, OUTLIER_VOLUME
                    ).reshape(-1)

                    if ALL_OBJECT_POSES[t-1].shape[0] != ALL_OBJECT_POSES[t-2].shape[0]:
                        prev_prev_poses =  ALL_OBJECT_POSES[t-1]
                    else:
                        prev_prev_poses =  ALL_OBJECT_POSES[t-2]


                    weights += prior_parallel_jit(
                        batch, ALL_OBJECT_POSES[t-1],  prev_prev_poses, renderer.model_box_dims, known_id
                    ).reshape(-1)

                    all_weights.append(weights)
                all_weights = jnp.hstack(all_weights)

                current_pose_estimate = all_pose_proposals[all_weights.argmax()]

            OBJECT_POSES = OBJECT_POSES.at[known_id].set(current_pose_estimate)

        # jprint("{}", OBJECT_POSES.shape)
        ##########################################################################################
        if OBJECT_POSES.shape[0] > 0:
            # jprint("Before render 2: {}", get_gpu_mem())
            rerendered = renderer.render(OBJECT_POSES, jnp.arange(OBJECT_POSES.shape[0]))[...,:3]
            # jprint("Before render 2: {}", get_gpu_mem())
            # jprint("{}", OBJECT_POSES[0])
        else:
            # jprint("Before copy branch: {}", get_gpu_mem())
            rerendered = jnp.copy(point_cloud_image)
            rerendered.at[...,-1].set(WALL_Z)
            # jprint("After copy branch: {}", get_gpu_mem())

        rerendered_spliced = splice_image_parallel(jnp.array([rerendered]), point_cloud_image_complement)[0]
        pixelwise_probs = gaussian_mixture_image_jit(point_cloud_image, rerendered_spliced, R_SWEEP)

        for seg_id in object_ids:
            average_probability = jnp.mean(pixelwise_probs[segmentation == seg_id])
            # print(seg_id, average_probability)

            if OBJECT_POSES.shape[0] > 0 and average_probability > 50.0:
                continue

            num_pixels = jnp.sum(segmentation == seg_id)
            if num_pixels < 14:
                continue

            rows, cols = jnp.where(segmentation == seg_id)
            distance_to_edge_1 = min(jnp.abs(rows - 0).min(), jnp.abs(rows - intrinsics.height).min())
            distance_to_edge_2 = min(jnp.abs(cols - 0).min(), jnp.abs(cols - intrinsics.width).min())

            point_cloud_segment = point_cloud_image[segmentation == seg_id]
            dims, pose = b.utils.aabb(point_cloud_segment)

            BUFFER = 3

            if distance_to_edge_1 < BUFFER or distance_to_edge_2 < BUFFER:
                continue

            resolution = 0.01
            voxelized = jnp.rint(point_cloud_segment / resolution).astype(jnp.int32)
            min_z = voxelized[:,2].min()
            depth = voxelized[:,2].max() - voxelized[:,2].min()

            front_face = voxelized[voxelized[:,2] <= min_z+20, :]
            slices = [front_face]
            for i in range(depth):
                slices.append(front_face + jnp.array([0.0, 0.0, i]))
            full_shape = jnp.vstack(slices) * resolution

            print("Seg ID: ", seg_id, "Prob: ", average_probability, " Pixels: ",num_pixels, " dists: ", distance_to_edge_1, " ", distance_to_edge_2, " Pose: ", pose[:3, 3])

            dims, pose = b.utils.aabb(full_shape)
            mesh = b.utils.make_marching_cubes_mesh_from_point_cloud(
                b.t3d.apply_transform(full_shape, b.t3d.inverse_pose(pose)),
                0.075
            )
            
            renderer.add_mesh(mesh)
            print("Adding new mesh")

            OBJECT_POSES = jnp.concatenate([OBJECT_POSES, pose[None, ...]], axis=0)
        ALL_OBJECT_POSES.append(OBJECT_POSES)

        self.ALL_OBJECT_POSES = ALL_OBJECT_POSES

        violations = []

        POSES = jnp.array(ALL_OBJECT_POSES[t])

        num_objects_previous = ALL_OBJECT_POSES[t-1].shape[0]
        num_objects_now = ALL_OBJECT_POSES[t].shape[0]
        num_new_objects = num_objects_now - num_objects_previous
        if num_new_objects > 0:
            print(f"{num_new_objects} new objects!")
        for new_object_index in range(num_objects_previous, num_objects_now):
            self.first_appearance.append(t)
            position = POSES[new_object_index,:3,3]
            ################################################################################################
            # jprint("Before render 3: {}", get_gpu_mem())
            rerendered = renderer.render(POSES[new_object_index:new_object_index+1], jnp.array([new_object_index]))[...,:3]
            # jprint("After render 3: {}", get_gpu_mem())
            rows, cols = jnp.where(rerendered[:,:,2] > 0.0)
            distance_to_edge_1 = min(jnp.abs(rows - 0).min(), jnp.abs(rows - intrinsics.height).min())
            distance_to_edge_2 = min(jnp.abs(cols - 0).min(), jnp.abs(cols - intrinsics.width).min())     
            if distance_to_edge_1 > 15 and  distance_to_edge_2 > 15:
                pixx, pixy= np.array(b.project_cloud_to_pixels(jnp.array([ position]), intrinsics).astype(jnp.int32)[0])
                for t_ in range(t-3):
                    occluded = b.utils.resize(images[t_].depth, intrinsics.height, intrinsics.width)[pixy, pixx] < position[2]
                    if not occluded:
                        pixx, pixy= np.array(b.project_cloud_to_pixels(jnp.array([ position]), intrinsics).astype(jnp.int32)[0]) * 4
                        violations.append({"x": pixx, "y": pixy})
                        plausibility -= 0.1
                        print("Object initialize not near edge! Implausiblepix!")
                        break

        for id_1 in range(num_objects_now):
            for id_2 in range(num_objects_now):
                if id_1 != id_2:
                    distance = jnp.linalg.norm(POSES[id_1,:3,3] - POSES[id_2,:3,3])
                    if distance < 0.4:
                        pixx, pixy= np.array(b.project_cloud_to_pixels(jnp.array([ POSES[id_1,:3,3]]), intrinsics).astype(jnp.int32)[0]) * 4
                        violations.append({"x": pixx, "y": pixy})
                        plausibility -= 0.1
                        print("Objects too close together! Implausible!")


        for id in range(num_objects_now):
            z_delta = POSES[id,:3,3][1] - ALL_OBJECT_POSES[self.first_appearance[id]][id,:3,3][1]
            if z_delta < -0.2:
                pixx, pixy=  np.array(b.project_cloud_to_pixels(jnp.array([ POSES[id,:3,3]]), intrinsics).astype(jnp.int32)[0]) * 4
                violations.append({"x": pixx, "y": pixy})
                plausibility -= 0.05
                print("Objects is not obeying gravity! Implausible!")

            if POSES[id,:3,3][1] > 1.5:
                pixx, pixy= np.array(b.project_cloud_to_pixels(jnp.array([ POSES[id,:3,3]]), intrinsics).astype(jnp.int32)[0]) * 4
                violations.append({"x": pixx, "y": pixy})
                plausibility -= 0.01
                print("Objects inside floor!")

            if POSES[id,:3,3][2] > 13.0:
                pixx, pixy= np.array(b.project_cloud_to_pixels(jnp.array([ POSES[id,:3,3]]), intrinsics).astype(jnp.int32)[0]) * 4
                violations.append({"x": pixx, "y": pixy})
                plausibility -= 0.01
                print("Objects behind wall!")

        self.plausibility.append(plausibility)
        self.violation_locations.append(violations)

    def process_message(self, message):
        (request_type, args) = message
        # print(f"received reqeust {request_type}")
        if request_type == "reset":
            (h,w,fx,fy,cx,cy,near,far) = args
            intrinsics = b.Intrinsics(
                h,w,fx,fy,cx,cy,near,far
            )
            self.reset(intrinsics)
            return True
        elif request_type == "update":
            self.update(args)
            return None
        elif request_type == "get_info":
            return self.plausibility[-1], self.violation_locations[-1]
        else:
            print("I HAVE NO IDEA WHAT REQUEST YOU'RE MAKING!")

In [9]:
def run_physics_scene(scene_name, physics_server):
    images = load_mcs_scene_data(scene_name)
    # print("Before server reset command -> mem: {}".format(get_gpu_mem()))
    physics_server.process_message(("reset",(400,600, 514.2991467983065,514.2991467983065,300.0,200.0,0.009999999776482582,150.0)))
    # print("Before first image -> mem: {}".format(0, get_gpu_mem()))
    for i, img in enumerate(images):
        # print(i+1)
        physics_server.process_message(("update", (img)))
        # print("After t: {}, mem: {}".format(i+1, get_gpu_mem()))
    plausibility, _ = physics_server.process_message(("get_info",None))

    physics_server.renderer.cleanup()

    print("Plausibility: ", plausibility)
    rating = plausibility > -0.001
    score = (1.0 / (1.0 + np.exp(-(plausibility + 0.01))))
    print(rating, score)
    return rating, physics_server
        

In [12]:
p = PhysicsServer()
_, p = run_physics_scene("passive_physics_shape_constancy_0001_05", p)

Before server reset command -> mem: 0.0


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Before first image -> mem: 0
After t: 1, mem: 1400.0
After t: 2, mem: 1400.0
After t: 3, mem: 1400.0
After t: 4, mem: 1400.0
After t: 5, mem: 1400.0
After t: 6, mem: 1400.0
After t: 7, mem: 1400.0
After t: 8, mem: 1400.0
After t: 9, mem: 1400.0
Seg ID:  5 Prob:  2596.2883  Pixels:  28  dists:  6   56  Pose:  [-1.0942516 -3.1375084  9.482194 ]
Centering mesh with translation [ 0.01500003 -0.01000004 -0.0100006 ]
Adding new mesh
1 new objects!
After t: 10, mem: 1632.0
After t: 11, mem: 5762.0
After t: 12, mem: 5762.0
After t: 13, mem: 5762.0
After t: 14, mem: 5762.0
After t: 15, mem: 5762.0
After t: 16, mem: 5762.0
After t: 17, mem: 5762.0
After t: 18, mem: 5762.0
After t: 19, mem: 5762.0
After t: 20, mem: 5762.0
After t: 21, mem: 5762.0
After t: 22, mem: 5762.0
After t: 23, mem: 5762.0
After t: 24, mem: 5762.0
After t: 25, mem: 5762.0
After t: 26, mem: 5762.0
After t: 27, mem: 5762.0
After t: 28, mem: 5762.0
After

: 

: 

In [9]:
# images = load_mcs_scene_data("passive_physics_shape_constancy_0001_01")
# display_mcs_video(images)

9457.0

In [10]:
phy_paths = "/home/ubuntu/physics_npzs/*.npz"
files = sorted(glob.glob(phy_paths))
files = [f.split("/")[-1].split(".")[0] for f in files]

# run full experiment
p = PhysicsServer()
result = {}
for i, f in enumerate(files):
    print("Before scene: {}", get_gpu_mem())
    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Scene #{}".format(i+1))
    result[f], p = run_physics_scene(f,p)
    print("After scene: {}", get_gpu_mem())



Before scene: {} 0.0
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Scene #1


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Seg ID:  0 Prob:  2921.9868  Pixels:  73  dists:  27   3  Pose:  [4.2027016 0.9983077 8.37962  ]
Centering mesh with translation [-0.01750001  0.01750001  0.00999928]
Adding new mesh
1 new objects!
destructor
resources released
Renderer resources and memory cleared.
Plausibility:  0.0
True 0.5024999791668749
After scene: {} 5363.0
Before scene: {} 5363.0
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Scene #2


[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Seg ID:  2 Prob:  2922.0771  Pixels:  73  dists:  27   3  Pose:  [4.202687   0.99830127 8.379562  ]
Centering mesh with translation [-0.01750001  0.01750001  0.00999928]
Adding new mesh
1 new objects!
destructor
resources released
Renderer resources and memory cleared.
Plausibility:  0.0
True 0.5024999791668749
After scene: {} 5365.0
Before scene: {} 5365.0
@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ Scene #3


...

Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Seg ID:  0 Prob:  4218.8306  Pixels:  214  dists:  21   8  Pose:  [-3.098012   1.1421766  6.9477973]
Centering mesh with translation [-0.00249991  0.00250003  0.00499983]
Adding new mesh
1 new objects!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Seg ID:  1 Prob:  12.223713  Pixels:  177  dists:  21   37  Pose:  [1.6240928 1.1411804 6.935071 ]
Centering mesh with translation [ 0.01249996  0.00250003 -0.01749982]
Adding new mesh
1 new objects!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside flo

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Seg ID:  1 Prob:  5625.827  Pixels:  207  dists:  19   7  Pose:  [-2.923517   1.1441506  6.4948177]
Centering mesh with translation [-0.0025001  -0.00249997  0.005     ]
Adding new mesh
1 new objects!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Seg ID:  2 Prob:  2.9471219  Pixels:  149  dists:  26   44  Pose:  [1.5047798 1.1303089 8.106952 ]
Centering mesh with translation [-0.00999995 -0.00249991  0.00749996]
Adding new mesh
1 new objects!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floor!
Objects inside floo

[E rasterize_gl.cpp:121] OpenGL version reported as 4.6


Increasing frame buffer size to (width, height, depth) = (160, 128, 1024)
Seg ID:  1 Prob:  4214.768  Pixels:  206  dists:  36   6  Pose:  [-3.1966872 -0.4114525  6.9479523]
Centering mesh with translation [-0.00750002 -0.01000001  0.00499983]
Adding new mesh
1 new objects!
Seg ID:  1 Prob:  8.462427e-27  Pixels:  188  dists:  25   33  Pose:  [1.802439   0.93208194 6.919019  ]
Centering mesh with translation [-0.01250002  0.00250003 -0.0175003 ]
Adding new mesh
1 new objects!
destructor
resources released
Renderer resources and memory cleared.
Plausibility:  0.0
True 0.5024999791668749
After scene: {} 9485.0


In [14]:
# import pickle
# with open('assets/baseline_april.pickle', 'wb') as file:
#     pickle.dump(result, file)

In [15]:
gt = np.loadtxt("../../../../eval_6_validation_ground_truth.csv",
                 delimiter=",", dtype=str)[31:105]

In [16]:
translate = {"implausible": False, "plausible": True}
total = 0
for i in range(gt.shape[0]):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
        print("Scene {}: Passed".format(gt[i][0]))
    else:
        print("Scene {}: Failed".format(gt[i][0]))

Scene passive_physics_collision_0001_01: Passed
Scene passive_physics_collision_0001_02: Passed
Scene passive_physics_collision_0001_03: Failed
Scene passive_physics_collision_0001_04: Passed
Scene passive_physics_collision_0001_05: Failed
Scene passive_physics_collision_0001_06: Failed
Scene passive_physics_collision_0001_13: Failed
Scene passive_physics_collision_0001_14: Failed
Scene passive_physics_collision_0001_15: Failed
Scene passive_physics_collision_0001_16: Failed
Scene passive_physics_collision_0001_17: Passed
Scene passive_physics_collision_0001_18: Passed
Scene passive_physics_gravity_support_0001_01: Passed
Scene passive_physics_gravity_support_0001_02: Passed
Scene passive_physics_gravity_support_0001_03: Failed
Scene passive_physics_gravity_support_0001_04: Passed
Scene passive_physics_gravity_support_0001_05: Failed
Scene passive_physics_gravity_support_0001_06: Failed
Scene passive_physics_gravity_support_0001_07: Failed
Scene passive_physics_gravity_support_0001_08:

In [17]:
total = 0
for i in range(0,12):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("Collision Passed: {}/{}, Accuracy: {}".format(total,12,total/12))
total = 0
for i in range(12,42):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("Gravity Passed: {}/{}, Accuracy: {}".format(total,30,total/30))
total = 0
for i in range(42,57):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("Object Permanence Passed: {}/{}, Accuracy: {}".format(total,15,total/15))
total = 0
for i in range(57,62):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("Shape Constancy Passed: {}/{}, Accuracy: {}".format(total,5,total/5))
total = 0
for i in range(62,74):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("Continuity Passed: {}/{}, Accuracy: {}".format(total,12,total/12))
for i in range(0,74):
    if result[gt[i][0]] == translate[gt[i][1]]:
        total += 1
print("---------------------------------------------------")
print("Total Passed: {}/{}, Accuracy: {}".format(total,74,total/74))

Collision Passed: 5/12, Accuracy: 0.4166666666666667
Gravity Passed: 16/30, Accuracy: 0.5333333333333333
Object Permanence Passed: 7/15, Accuracy: 0.4666666666666667
Shape Constancy Passed: 2/5, Accuracy: 0.4
Continuity Passed: 11/12, Accuracy: 0.9166666666666666
-------------------------------------------------s
Total Passed: 52/74, Accuracy: 0.7027027027027027
